In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

! pip install conda install pytorch torchvision torchaudio pytorch-cuda=11.8 -c pytorch -c nvidia
! pip install torch==2.5.1
! pip install bitsandbytes==0.3.0
! pip install transformers==4.49.0
! python3 visualization_miniGPT4.py --cfg-path '/home2/palash/p1_Jailbreak/MiniGPT4/eval_configs/minigpt4_eval.yaml'

In [ ]:
import sys
path = "/home2/palash/p1_Jailbreak/MiniGPT4/common"
sys.path.append(path)

import os, re, gc, json, argparse, random, torch
from collections import defaultdict
import numpy as np
from PIL import Image
from tqdm import tqdm
from torch.utils.data import DataLoader
from minigpt4.conversation.conversation import Chat
from minigpt4.common.config import Config
from minigpt4.common.eval_utils import prepare_texts, init_model, eval_parser, computeIoU

In [ ]:
image_path = f'/home2/palash/p1_Jailbreak/LLaVA-NeXT/images/Memes/BENIGN_visual/benign_363.jpg'
images = Image.open(image_path)
prompt = "[INST] <image>[/INST]"
inputs = processor(images, prompt, return_tensors="pt").to(device)
inputs = {k: v.to(device) for k, v in inputs.items()}


# outputs = model.generate(
#         **inputs, 
#         max_new_tokens=1,
#         pad_token_id=model.config.pad_token_id,
#         eos_token_id=model.config.eos_token_id,
#         return_dict_in_generate=True,
#         output_attentions=True,
#         )

In [ ]:
prompt = "<image>"
inputs = processor(prompt, return_tensors="pt").to(device)
print( inputs['input_ids'][0] )

In [ ]:
import torch

def generate_output(prompt, images, model, processor, device_id, max_new_tokens=256):
    
    # prompt = "[INST] <image>[/INST]"
    # image = Image.open(image_path)
    # inputs = processor(images=image, text=prompt, return_tensors="pt")
    inputs = processor(images, prompt, return_tensors="pt").to(device_id)
    inputs = {k: v.to(device) for k, v in inputs.items()}

    outputs = model.generate(
        **inputs, 
        max_new_tokens=max_new_tokens,
        pad_token_id=model.config.pad_token_id,
        eos_token_id=model.config.eos_token_id,
        return_dict_in_generate=True,
        output_attentions=True,
        )
    # processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
    attentions = [[att.detach().cpu() for att in attn] for attn in outputs['attentions']]
    sequences = outputs['sequences'].detach().cpu()  # Move sequences to CPU
    past_key_values = [[[kv.detach().cpu() for kv in layer] for layer in pkv] for pkv in outputs['past_key_values']]
    
    print(f'inputs: {inputs["input_ids"]}')
    with torch.no_grad():  # Ensure no gradients are computed
        # op = model(**inputs)
        # print(op.keys())
        image_token_mask = (inputs["input_ids"] == 32000).int().detach().cpu()
        # image_token_mask = (model(**inputs).image_token_mask).detach().cpu()
        
    return (attentions, image_token_mask)



In [ ]:
def compute_average_cross_attention(attentions, image_token_mask):
    """
    Computes the average cross-attention given to image tokens by text tokens.
    
    Parameters:
    - attentions: Tensor of (num_layers, batch_size, num_heads, seq_len (query), seq_len (key))
    - image_token_mask: Tensor of shape (batch_size, seq_len), where True indicates image tokens.
    
    Returns:
    - avg_attention_per_layer: Tensor of shape (num_layers, num_heads), average attention to image tokens.
    """
    num_layers, batch_size, num_heads, seq_len, _ = attentions.shape
    # Shape: (32, 1, 32, 586, 586)

    # Remove batch dimension
    attentions = attentions.squeeze(1)  # Shape: (32, 32, 586, 586)

    # Identify image and text tokens
    image_token_indices = torch.where(image_token_mask.squeeze(0))[0]  # Indices of image tokens
    text_token_indices = torch.where(~image_token_mask.squeeze(0))[0]  # Indices of text tokens

    # Get attention from text tokens to image tokens: (num_layers, num_heads, num_text_tokens, num_image_tokens)
    text_to_image_attn = attentions[:, :, text_token_indices, :][:, :, :, image_token_indices]

    # Average across text tokens, image tokens and heads: Shape (num_layers)
    avg_attention_per_layer = text_to_image_attn.mean(dim=(1, 2, 3))
    
    return avg_attention_per_layer  # Shape: (num_layers)




In [ ]:
print("All special tokens:", processor.tokenizer.all_special_tokens)
print(processor.tokenizer('<image>'))

In [ ]:
import os

path_benignImages = f'/home2/palash/p1_Jailbreak/LLaVA-NeXT/images/Memes/BENIGN_visual/'
path_toxicImages = f'/home2/palash/p1_Jailbreak/LLaVA-NeXT/images/Memes/TOXIC_visuals/'
path_memeImages = f'/home2/palash/p1_Jailbreak/LLaVA-NeXT/images/Memes/ISLAM_memes/'


all_benignImages = os.listdir(path_benignImages)
print(all_benignImages)

all_toxicImages = os.listdir(path_toxicImages)
print(all_toxicImages)

all_memeImages = os.listdir(path_memeImages)
print(all_memeImages)


query_list = []
with open(f'../jb_dataset/BeaverTails/BeaverTailsTestCases_100.txt', 'r') as f:
        queries = f.readlines()
        query_list = [i.replace('\n', '') for i in queries]

print(f'*** A total of {len(query_list)} queries.')


In [ ]:
import random

benign_list = []
toxic_list = []
memes_list = []

for idx, ip in enumerate( query_list):
    
    prompt = f"<image>\nUSER: Answer the following question:\n{ip} ASSISTANT:" 
    
    path = path_benignImages + random.choice(all_benignImages)
    benign_images = [Image.open(path).convert('RGB')]

    path = path_toxicImages + random.choice(all_toxicImages)
    toxic_images = [Image.open(path).convert('RGB')]

    path = path_memeImages + random.choice(all_memeImages)
    meme_images = [Image.open(path).convert('RGB')]

    benign_list.append((prompt, benign_images))
    toxic_list.append((prompt, toxic_images))
    memes_list.append((prompt, meme_images))
    


In [ ]:
print( len(benign_list))
print( len(toxic_list))
print( len(memes_list))

In [ ]:
benign_list[:3]

In [ ]:
benign_attns = []

# Process benign inputs
i=0
for prompt, image in benign_list:
    i+=1
    print(i)
    attn , i_t_m = generate_output(prompt, image, model, processor, device, max_new_tokens=1)
    attn = torch.stack(attn[0])
    attn = compute_average_cross_attention(attn, i_t_m)
    benign_attns.append(attn)
    print(f"Output of benign prompt {i} generated")

torch.save(benign_attns,"benign_attention_weights.pt")
benign_attns = [] # Clear the weights


In [ ]:
toxic_attns = []

# Process toxic inputs
i=0
for prompt, image in toxic_list:
    i+=1
    attn , i_t_m = generate_output(prompt, image, model, processor, device, max_new_tokens=1)
    attn = torch.stack(attn[0])
    attn = compute_average_cross_attention(attn, i_t_m)
    toxic_attns.append(attn)
    print(f"Output of toxic prompt {i} generated")

torch.save(toxic_attns,"toxic_attention_weights.pt")
toxic_attns = [] # Clear the weights


In [ ]:
memes_attns = []

# Process memes inputs
i=0
for prompt, image in memes_list:
    i+=1
    print(i)
    attn , i_t_m = generate_output(prompt, image, model, processor, device, max_new_tokens=1)
    attn = torch.stack(attn[0])
    attn = compute_average_cross_attention(attn, i_t_m)
    memes_attns.append(attn)
    print(f"Output of memes prompt {i} generated")

torch.save(memes_attns,"memes_attention_weights.pt")
memes_attns = [] # Clear the weights


In [ ]:
import matplotlib.pyplot as plt

def plot_attention_scores(toxic_attentions, benign_attentions, name):
    """
    Plots attention scores for toxic and benign inputs.

    Args:
        toxic_attentions (list of torch.Tensor): List of attention tensors for toxic inputs.
        benign_attentions (list of torch.Tensor): List of attention tensors for benign inputs.
    """

    
    # Convert lists of tensors to a single tensor for easier manipulation
    toxic_attentions_tensor = torch.stack(toxic_attentions)  # Shape: [num_samples, 32]
    benign_attentions_tensor = torch.stack(benign_attentions)  # Shape: [num_samples, 32]

    # Number of layers
    num_layers = toxic_attentions_tensor.shape[1]

    toxic_attentions_tensor = toxic_attentions_tensor.mean(dim=0) # Shape: [32]
    benign_attentions_tensor = benign_attentions_tensor.mean(dim=0) # Shape: [32]

    # Create a scatter plot for each layer
    plt.figure(figsize=(12, 6))
    # for layer in range(num_layers):
    #     # Plot toxic attention scores
    #     plt.scatter(
    #         [layer] * len(toxic_attentions_tensor),  # X-axis: layer index
    #         toxic_attentions_tensor[:, layer],  # Y-axis: attention scores
    #         color='red',  # Color for toxic points
    #         label='Toxic' if layer == 0 else None,  # Label only once
    #         alpha=0.6  # Transparency
    #     )
    #     # Plot benign attention scores
    #     plt.scatter(
    #         [layer] * len(benign_attentions_tensor),  # X-axis: layer index
    #         benign_attentions_tensor[:, layer],  # Y-axis: attention scores
    #         color='blue',  # Color for benign points
    #         label='Benign' if layer == 0 else None,  # Label only once
    #         alpha=0.6  # Transparency
    #     )
    layers = list(range(1, num_layers+1))
    plt.plot(layers, benign_attentions_tensor, marker='s', linestyle='none', label='Benign Image', color='blue')
    plt.plot(layers, toxic_attentions_tensor, marker='s', linestyle='none', label='Meme Image', color='red')

    # Customize the plot
    plt.xlabel('Layer Index')
    plt.ylabel('Attention Score')
    plt.title('Average Cross Attention Scores Across Layers')
    plt.xticks(range(1, num_layers+1))  # Set x-ticks to layer indices
    plt.legend()  # Show legend
    plt.grid(True, linestyle='--', alpha=0.6)  # Add grid lines
    plt.savefig(f'attention_scores_for{name}.png')  # Save the plot as a PNG file
    print("Plot saved")

In [ ]:
# Visualize
print("Visualizing Average Cross Attentions")

toxic_attns = torch.load("./toxic_attention_weights.pt")
benign_attns = torch.load("./benign_attention_weights.pt")
    
plot_attention_scores(toxic_attns, benign_attns, 'benVsToxic')

In [ ]:
# Visualize
print("Visualizing Average Cross Attentions")

memes_attns = torch.load("./memes_attention_weights.pt")
benign_attns = torch.load("./benign_attention_weights.pt")
    
plot_attention_scores(memes_attns, benign_attns, 'benVsMemes')

In [ ]:

toxic_list = []
benign_list = []

image_paths = ["jb_memes/benign_5.jpg", "jb_memes/benign_2.jpg", "jb_memes/benign_3.jpg", "jb_memes/benign_4.jpg", "jb_memes/benign_1.jpg"]
benign_images = [Image.open(image_path).convert('RGB') for image_path in image_paths]

with open("jb_outputs_refined.csv", 'r', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    next(reader) # Skip the header

    for row in reader:
        text_input, benign_output, jb_output, meme_images_str = row
        meme_images = [meme_image.strip() for meme_image in meme_images_str.split(";")]
        meme_images = [Image.open(meme_image).convert('RGB') for meme_image in meme_images]
        prompt = f"USER: <image>\n<image>\n<image>\n<image>\n<image>\nAnswer the following question:\n{text_input} ASSISTANT:"
        toxic_list.append((prompt, meme_images))
        benign_list.append((prompt, benign_images))

# Debugging
benign_list = benign_list[20:40]
toxic_list = toxic_list[20:40]



In [ ]:
toxic_attns = []
benign_attns = []

# Process toxic inputs
i=0
for prompt, image in toxic_list:
    i+=1
    attn , i_t_m = generate_output(prompt, image, model, processor, device_id)
    attn = torch.stack(attn[0])
    attn = compute_average_cross_attention(attn, i_t_m)
    toxic_attns.append(attn)
    print(f"Output of toxic prompt {i} generated")

torch.save(toxic_attns,"toxic_attention_weights_20-39.pt")
toxic_attns = [] # Clear the weights
